In [ ]:
import sys
from IPython.display import Image, display
import sympy
sys.path.append('..')   # path for local package
import restflow
from restflow import symvec, symtools

# Model B equation

The Model B equation is given by:
\begin{equation*}
\partial_t\phi = \nabla^2(a\phi+u\phi^3-\kappa\nabla^2\phi)  + \eta,
\end{equation*}
with conserved noise. The quantity $\phi$ is usually related to the density. For more on the physics of Model B, see Section I.1 from the accompanied manuscript.

We will only consider 1-loop graphs. For more loops, their contributions are of higher order in the perturbation series. 

<b><ins>1st Step:</b></ins> Define the model parameters, the vectors, the propagators and the vertex functions:

In [ ]:
# define the model
class modelb:
    def __init__(self):
        self.alpha = 2
        self.kap, self.D, self.u, self.a, self.bbar = sympy.symbols('kappa D u a b_b')

    def f(self,k):
        return (self.kap*k**2+self.a)*k**2

    def v3(self,k1,k2,k3,q):
        return (-self.u*q**2,1)

# symbols for vectors
_q, _k, dot_kq = sympy.symbols('q k (k·q)')
# saves the vectors and dot products in a class
ctx = symvec.Context()
ctx.add_dot_product(_q,_k,dot_kq)
ctx = restflow.Context()
ctx.add_dot_product(_q,_k,dot_kq)
# create symbolic wave vectors
k = ctx.vector(_k)
q = ctx.vector(_q)

model = modelb()

# Calculation of graphs

### Graphical correction for propagator

The only one-loop graph that contributes to the propagator is

In [ ]:
pil_img = Image(filename='./figures/modelB_1vertex.jpg')
display(pil_img)

In [ ]:
labels = [k, q]
v = [restflow.Vertex() for i in range(2)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[1])
v[0].add_outgoing(0.0)
g = restflow.Graph(v)
g.label_edges(labels)
expr = g.convert(model)
I1 = restflow.integrate([expr],5,labels)
display(I1)

### Graphical correction for 3-vertex

The only one-loop graph that contributes to the 3-vertex is

In [ ]:
pil_img = Image(filename='./figures/modelB_3vertex.jpg')
display(pil_img)

In [ ]:
# define the external legs p and r. They will be set to 0 since the 3-vertex is independent of them.
_p, dot_pk, dot_qp, _r, dot_qr, dot_pr, dot_rk  = sympy.symbols('p (k·p) (q·p) r (q·r) (p·r) (r·k)')
p = ctx.vector(_p)
r = ctx.vector(_r)
ctx.add_dot_product(_q,_k,dot_kq)
ctx.add_dot_product(_q,_p,dot_qp)
ctx.add_dot_product(_p,_k,dot_pk)
ctx.add_dot_product(_q,_r,dot_qr)
ctx.add_dot_product(_p,_r,dot_pr)
ctx.add_dot_product(_r,_k,dot_rk)
labels = [k, p, r, q-p-r]
# figure (3c)
v = [restflow.Vertex() for i in range(3)]
v[0].link_vertex(v[1])
v[0].link_vertex(v[2])
v[2].link_vertex(v[1])
v[0].add_outgoing()
v[2].add_outgoing()
v[2].add_outgoing()
g = restflow.Graph(v)
exprs = g.convert_perm(model,labels)
exprs = symtools.nullify_outlegs(exprs, k, q, p, r)
I3 = restflow.integrate(exprs,3,labels)
display(I3)


# Graphical corrections and dimensionless parameters

Once we have the corrections to the propagator and the 3-vertex, we can renormalize the model parameters by comparing the coefficients of the renormalized and unrenormalized vertex functions. Then, by proper scaling to restore the cut-off, the Wilson's flow equations can be found (see section II.G.). 

From the $I_1$ we see that there is no term of order $q^4$ and therefore $\psi_\kappa=0$. The first correction to $D$ is a two-loop integral and thus of order $(\delta l)^2$ with $\psi_D=0$. 

We now calculate the graphical corrections for $\psi_a$ and $\psi_u$:

In [ ]:
delta_l = sympy.symbols('δl')
psi_a = sympy.simplify(-sympy.simplify(I1/q**2/delta_l)/model.a)
psi_u = sympy.simplify(sympy.Poly(sympy.simplify(-I3/q.sym**2/delta_l)).as_expr()/model.u)
display(psi_a)
display(psi_u)

We define the following dimensionless parameters:
\begin{gather}
\bar{a} \equiv \frac{a}{\kappa \Lambda^2}, \bar{u}\equiv\frac{uD}{\kappa^2}K_d \Lambda^{d-4}
\end{gather}

We now express $\psi_a$ and $\psi_u$ wrt $\bar u$ and $\bar a$:

In [ ]:
d, Kd, Lambda, abar, ubar = sympy.symbols('d K_d Lambda a_b u_b')

psi_a = sympy.simplify(psi_a.subs([(model.a, abar*model.kap*Lambda**2), (model.u, ubar*model.kap**2/model.D*Lambda**(-d+4)/Kd)]))
psi_u = sympy.simplify(psi_u.subs([(model.a, abar*model.kap*Lambda**2), (model.u, ubar*model.kap**2/model.D*Lambda**(-d+4)/Kd)]))

display(psi_a)
display(psi_u)

Thus the graphical corrections are $\psi_D=0$ and $\psi_a = \frac{3\bar{u}}{\bar{a}(\bar{a}+1)}$ and $\psi_u = -\frac{9\bar{u}}{(\bar{a}+1)^2}$.

Using dimensional arguments we can now obtain the flow equation of $\bar a$ and $\bar u$:

In [ ]:
flow_a, flow_u = 2*abar+psi_a*abar, (4-d)*ubar+psi_u*ubar
display(flow_a)
display(flow_u)

From this it is straightforward to obtain the non-trivial fixed points with respect to $\epsilon=4-d$.

In [ ]:
from IPython.display import display, Math

epsilon = sympy.symbols('\epsilon')
flow_a = 2*abar+psi_a*abar
flow_u = (4-d)*ubar+psi_u*ubar
sol = sympy.solve([flow_a, flow_u], [abar, ubar])
sol = [[sympy.series(element[0].subs(d,4-epsilon),epsilon,0,2).removeO(), sympy.series(element[1].subs(d,4-epsilon),epsilon,0,2).removeO(), ] for element in sol]
display(Math(r'Gaussian: \bar a={}, \bar u = {} \\ Wilson-Fisher: \bar a={}, \bar u={}'.format(sol[0][0], sol[0][1],sol[1][0],sol[1][1])))


We can also calculate eigenvectors and eigenvalues of the linearized system around the Gaussian and the Wilson-Fisher  fixed points:

In [ ]:
def calculate_theor_stability(pt):
  '''
  Calculates symbolically the eigenvectors and eigenvalues given flow equations and a point.
  '''
  flows = [sympy.series(element.subs(d,4-epsilon), epsilon,0,2).removeO() for element in [flow_a, flow_u]]
  J= (sympy.Matrix(flows).jacobian([abar,ubar])).subs([(abar,pt[0]), (ubar,pt[1])])
  for i in range(2):
    for j in range(2):
      J[i,j] = sympy.series(J[i,j],epsilon,x0=0,n=2).removeO()
  eigenvec = J.eigenvects()
  eigenpair = [[eigenvec[i][0], eigenvec[i][2][:]] for i in range(len(eigenvec))]
  for i in range(len(eigenvec)):
    for j in range(len(eigenpair[i][1])):
      eigenpair[i][0] = sympy.series(eigenpair[i][0],epsilon,x0=0,n=2).removeO()
      eigenpair[i][1][j] = sympy.Matrix([sympy.N(sympy.series(element,epsilon,x0=0,n=2).removeO()) for element in eigenpair[i][1][j]])
  return [eigenpair[i][0] for i in range(len(eigenpair))], [eigenpair[i][1][0] for i in range(len(eigenpair))]

elam_g, evec_g = calculate_theor_stability([sol[0][0], sol[0][1]])
elam_wf, evec_wf = calculate_theor_stability([sol[1][0], sol[1][1]])
for i in range(2):
  display(Math(r'Gaussian: eigenvalue \quad \lambda_{}^G={}, \quad eigenvector \quad v^G_{}= '.format(i,elam_g[i],i)))
  display(evec_g[i])
for i in range(2):
  display(Math(r'Wilson-Fisher: eigenvalue \quad \lambda_{}^W={}, \quad eigenvector \quad v^W_{}= '.format(i,elam_wf[i],i)))
  display(evec_wf[i])

Therefore, for $d<4$, the flow is repulsive close to the Gaussian while for the Wilson-Fisher the flow is attractive along the direction of $u_0^W$ and repulsive along the $\bar a$-axis. While this is valid only close to $d=4$, there is ample of evidence that the Wilson-Fisher controls the behavior down to $d=2$ <sup>1</sup>.

# References

<sup>1</sup> J. Le Guillou and J. Zinn-Justin, “Accurate critical ex-
ponents from the ε-expansion,” J. Physique Lett. 46,
137–141 (1985).